In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2
import os
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
IM_SIZE = 200
TRAIN_PATH = r'./train'
TEST_PATH = r'./test'
def read_data(path):
    X = []
    
    files = os.listdir(path)
    
    for file in files:
        
        image = imread(path+ '\\' + file)
        image_resized = cv2.resize(image, (IM_SIZE, IM_SIZE))
        X.append(image_resized)

    return np.array(X)

x_test = read_data(r'.\test')
x_train = read_data(r'.\train')

# Procesar las imagenes tal y como entran en el modelo
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

print(x_train.shape)
print(x_test.shape)

(9000, 200, 200, 3)
(1150, 200, 200, 3)


In [3]:
train_df = pd.read_csv(r'.\\file_target.csv')

train_df = train_df.rename(columns={'archivo': 'filenames',
                       'target': 'category'})

train_df

,filenames,category
0,0.jpg,G
1,1.jpg,H
2,2.jpg,G
3,3.jpg,X
4,4.jpg,H
...,...,...
8995,8995.jpg,G
8996,8996.jpg,A
8997,8997.jpg,L
8998,8998.jpg,E


In [4]:
test_files = os.listdir('./test')

print(type(test_files))

test_df = pd.DataFrame(test_files, columns = ['filenames'])

test_df

<class 'list'>


,filenames
0,0.jpg
1,1.jpg
2,10.jpg
3,100.jpg
4,1000.jpg
...,...
1145,995.jpg
1146,996.jpg
1147,997.jpg
1148,998.jpg


In [5]:
train, validate = train_test_split(train_df, test_size = 0.18)

In [6]:
train = train.reset_index(drop=True)

train.head()

,filenames,category
0,1280.jpg,O
1,8836.jpg,Z
2,7909.jpg,X
3,4422.jpg,C
4,1665.jpg,W


In [7]:
validate = validate.reset_index(drop=True)

validate.head()

,filenames,category
0,7972.jpg,R
1,5166.jpg,E
2,3169.jpg,P
3,5825.jpg,O
4,6777.jpg,K


In [8]:
train_datagen = ImageDataGenerator(rescale = 1.0/255.0)

validation_datagen = ImageDataGenerator(rescale = 1.0/255.)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(train,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 50,
                                                    class_mode = 'categorical',
                                                    target_size = (IM_SIZE, IM_SIZE))

validation_generator = validation_datagen.flow_from_dataframe(validate,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 50,
                                                              class_mode = 'categorical',
                                                              target_size = (IM_SIZE, IM_SIZE))

prediction_data_generator = test_datagen.flow_from_dataframe(test_df,
                                                              TEST_PATH,
                                                              x_col='filenames',
                                                              batch_size = 50,
                                                              target_size = (IM_SIZE, IM_SIZE))

In [ ]:
base_model = ResNet50V2(input_shape=(IM_SIZE, IM_SIZE, 3),
                       classifier_activation='softmax',
                       include_top=False)

for layer in base_model.layers:
    layer.trainable=False

#### FULLY CONNECTED LAYER ####
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [ ]:
rentv2 = model.fit(train_generator,
                    validation_data = validation_generator,
                   batch_size = 10,
                    epochs = 1,
                  steps_per_epoch=10)

In [ ]:
predictions = model.predict(test_generator)

In [ ]:
predictions.shape